In [2]:
import csv
import requests
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os 

# import the webdriver, chrome driver is recommended
driver = webdriver.Chrome(ChromeDriverManager().install())
cwd = os.getcwd()
   
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True
    time.sleep(2)

#Function to check if there are any reviews that need to click on the 'more info.. / keep reading' web elements
def more():
    snippet = driver.find_element_by_class_name('postSnippet')
    if len(snippet)>0:
        return True
    else:
        return False 
    

pages_to_scrape = 3 # <-- This depends on the overall pages that the website has, there are ways to automate this input but I just look at the driver browser pop up and write down how many pages I need to scrape :S 
url = 'https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d10397390-Reviews-M_S_Cafe-London_England.html'
driver.get(url)
#to maximize the the tab 
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\LanceTersou\.wdm\drivers\chromedriver\win32\99.0.4844.51]
C:\Users\LanceTersou\miniconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':


In [3]:
# open the file to save the review
path_to_file = cwd+"\FILENAME.csv"
csvFile = open(path_to_file, 'a', encoding="utf-8-sig")
csvWriter = csv.writer(csvFile)

In [ ]:
# change the value inside the range to save the number of reviews we're going to grab
for i in range(0, pages_to_scrape):

    # give the DOM time to load
    time.sleep(2) 

    # Click the "expand review" link to reveal the entire review.
    driver.find_element_by_xpath(".//div[contains(@class, 'widgetEvCall')]").click()

    # Now we'll ask Selenium to look for elements in the page and save them to a variable. First lets define a  container that will hold all the reviews on the page. In a moment we'll parse these and save them:
    container = driver.find_elements_by_xpath("//div[@data-reviewid]")

    # Next we'll grab the date of the review:
    dates = driver.find_elements_by_xpath(".//span[contains(@class, 'ratingDate')]")
    
   # Now we'll look at the reviews in the container and parse them out

    for j in range(len(container)): # A loop defined by the number of reviews

        # Grab the rating
        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        # Grab the title
        title = container[j].find_element_by_xpath(".//span[contains(@class, 'noQuotes')]").text
        #Grab the review
        review = container[j].find_element_by_xpath(".//div[contains(@class, 'entry')]").text.replace("\n", "  ") 
        #Grab the data
        date = " ".join(dates[j].text.split(" ")[-2:])
        
        #Save that data in the csv and then continue to process the next review
        csvWriter.writerow([ date, rating, title, review])# postSnippet]) 
        
    # When all the reviews in the container have been processed, change the page and repeat            
    driver.find_element_by_xpath('.//a[@class="ui_button nav next primary "]').click()

# When all pages have been processed, quit the driver
driver.quit()